# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [37]:
from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
!pip install  fr_core_news_md
import os

In [38]:
year = "1949"
txt_path = '../../data/txt'
with open(f"../../data/all_{year}.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(txt_path):
        if file.endswith(".txt")and year in file:
            with open(os.path.join(txt_path, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())


In [39]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [40]:
# Imprimer le corpus de Spacy
sentences

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs",
 "San Francisco envisage d'interdire les robots coursiers sur les trottoirs",
 'Londres est une grande ville du Royaume-Uni',
 'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe',
 "Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon",
 "La France ne devrait pas manquer d'électricité cet été, même en cas de canicule",
 'Nouvelles attaques de Trump contre le maire de Londres',
 'Où es-tu ?',
 'Qui est le président de la France ?',
 'Où est la capitale des États-Unis ?',
 'Quand est né Barack Obama ?']

In [41]:
# Isoler la première phrase
sent = sentences[0]
sent

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [42]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [43]:
type(doc)

spacy.tokens.doc.Doc

In [44]:
doc.text

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [45]:
doc.to_json()

{'text': 'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 'ents': [{'start': 0, 'end': 5, 'label': 'ORG'}],
 'sents': [{'start': 0, 'end': 72}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'tag': 'PROPN',
   'pos': 'PROPN',
   'morph': 'Gender=Masc|Number=Sing',
   'lemma': 'Apple',
   'dep': 'nsubj',
   'head': 1},
  {'id': 1,
   'start': 6,
   'end': 13,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
   'lemma': 'cherche',
   'dep': 'ROOT',
   'head': 1},
  {'id': 2,
   'start': 14,
   'end': 15,
   'tag': 'ADP',
   'pos': 'ADP',
   'morph': '',
   'lemma': 'à',
   'dep': 'mark',
   'head': 3},
  {'id': 3,
   'start': 16,
   'end': 23,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'VerbForm=Inf',
   'lemma': 'acheter',
   'dep': 'xcomp',
   'head': 1},
  {'id': 4,
   'start': 24,
   'end': 27,
   'tag': 'DET',
   'pos': 'DET',
   'morph': 'Definite=Ind|Gender=Fem|Number=Sing|PronType=Art'

In [46]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars' contient les entités suivantes : Apple (ORG)
'Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs' ne contient aucune entité
'San Francisco envisage d'interdire les robots coursiers sur les trottoirs' contient les entités suivantes : San Francisco (LOC)
'Londres est une grande ville du Royaume-Uni' contient les entités suivantes : Londres (LOC), Royaume-Uni (LOC)
'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe' contient les entités suivantes : L’Italie (LOC), ArcelorMittal (ORG), Europe (LOC)
'Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon' contient les entités suivantes : Apple (ORG), HomePod (MISC), Echo (ORG), Amazon (ORG)
'La France ne devrait pas manquer d'électricité cet été, même en cas de canicule' contient les entités suivantes : La France (LOC)
'Nouvelles attaques de Trump contre le maire de Londres' contient les en

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [47]:
# Charger le texte
n=1000000
text = open(f"../../data/all_{year}.txt", encoding='utf-8').read()[:n]

In [48]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: user 27.6 s, sys: 1.4 s, total: 29 s
Wall time: 29.4 s


In [49]:
# Compter les entités
people = defaultdict(int)
orgs = defaultdict(int)
locs=defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    elif ent.label_ == "ORG":
        orgs[ent.text] += 1
    elif ent.label_ == "LOC":
        locs[ent.text] += 1

In [50]:
# Trier et imprimer
# Trier et afficher les personnes
sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

# Trier et afficher les organisations
sorted_orgs = sorted(orgs.items(), key=lambda kv: kv[1], reverse=True)
for org, freq in sorted_orgs[:20]:
    print(f"Organisation: {org} apparait {freq} fois")

# Trier et afficher les lieux
sorted_locs = sorted(locs.items(), key=lambda kv: kv[1], reverse=True)
for loc, freq in sorted_locs[:20]:
    print(f"Lieu: {loc} apparait {freq} fois")

Rossel apparait 299 fois dans le corpus
Ecrire Ag apparait 30 fois dans le corpus
Louise apparait 25 fois dans le corpus
A.F.P. apparait 19 fois dans le corpus
trav apparait 17 fois dans le corpus
Mutuel apparait 15 fois dans le corpus
Pronostics apparait 13 fois dans le corpus
Pauwels apparait 13 fois dans le corpus
M. Van Cauwelaert apparait 12 fois dans le corpus
- Tél apparait 12 fois dans le corpus
Linssen apparait 11 fois dans le corpus
Régent apparait 10 fois dans le corpus
Inut apparait 10 fois dans le corpus
Notaire apparait 10 fois dans le corpus
M. Vychinsky apparait 9 fois dans le corpus
Agence Rossel apparait 9 fois dans le corpus
M. Bréhier apparait 9 fois dans le corpus
Gérardin apparait 9 fois dans le corpus
Anne-Marie apparait 8 fois dans le corpus
Jette apparait 8 fois dans le corpus
s.d.b apparait 8 fois dans le corpus
chamb apparait 8 fois dans le corpus
Molière apparait 8 fois dans le corpus
Reiff apparait 8 fois dans le corpus
Washer apparait 8 fois dans le corpus

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus